# CPMM Demo: A Toy Liquidity Pool

Owner: April Nellis

Companion code to [*DEX Specs: A Mean-Field Approach to DeFi Cryptocurrency Exchanges*](https://arxiv.org/abs/2404.09090)

In [ ]:
import numpy as np
import pandas as pd
import math
import itertools
pd.options.mode.chained_assignment = None  # default='warn'
import os.path
from datetime import datetime
import time
import requests
import statsmodels.api as sm
import scipy.stats as stats
import scipy.optimize as optimize
from IPython.display import Image
from coinmetrics.api_client import CoinMetricsClient
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation
import import_ipynb
from IPython.display import IFrame
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Just disables the annoying warning, doesn't enable AVX/FMA
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

%matplotlib widget

plt.rcParams["figure.autolayout"] = True

In [ ]:
plt.rcParams["figure.figsize"] = [5,5]

## Defining The Structure of the Simulation

In [ ]:
%run DEX.py

#### Simple demonstration of some features

A simple demonstration of setting and updating liquidity in the pool.

In [ ]:
liq = Pool(pool_type = 1, mkt =1.5, pool =1.5, fee = 0.01)

unif = np.ones(liq.size)*10 # uniform value in terms of 1.6 B/A
equalL = np.array([10, 10, 9.9, 8.79, 7.62, 20/3]) # uniform ell
variety = np.array([7, 9, 11, 10, 8, 6]) # nice rounded plot

liq.resetLiquidity(variety, 1.6)
liq.printDetails()

In [ ]:
liq.plotLiquidity(yLim = [8, 12])

In [ ]:
ell = np.zeros(liq.size)
for i in range(liq.size):
    ell[i] = liq.K(i)
print(ell)
liq.plotEll(ell, yLim = 150)

In [ ]:
a_list, b_list = liq.calculatePsi()

In [ ]:
oldA = np.copy(liq.tokA)
oldB = np.copy(liq.tokB)

### Demonstration of Liquidity Addition

In [ ]:
liq_add = 10 # we want to add one unit of liquidity to a few ticks

In [ ]:
def liq_to_capital(u, idx, aNu): # units of liquidity to add, tick to add to, liquidity pool
    if idx == aNu.idx:
        return u * aNu.mkt_er * (1/math.sqrt(aNu.pool_er) - 1/math.sqrt(aNu.ticks[idx+1])) + u*(math.sqrt(aNu.pool_er) - math.sqrt(aNu.ticks[idx]))
    elif idx < aNu.idx:
        return u*(math.sqrt(aNu.ticks[idx+1]) - math.sqrt(aNu.ticks[idx]))
    else:
        return u*aNu.mkt_er*(1/math.sqrt(aNu.ticks[idx]) - 1/math.sqrt(aNu.ticks[idx+1]))

In [ ]:
liq.updateLiq(1, liq_to_capital(liq_add, 1, liq))
liq.updateLiq(2, liq_to_capital(liq_add, 2, liq))
liq.updateLiq(3, liq_to_capital(liq_add, 3, liq))
ell2 = np.zeros(liq.size)
for i in range(liq.size):
    ell2[i] = liq.K(i)

In [ ]:
ell2 - ell

In [ ]:
plt.close()
curr_er = liq.idx + (liq.pool_er - liq.ticks[liq.idx])/(liq.ticks[liq.idx + 1] - liq.ticks[liq.idx]) - 1

plt.bar(np.arange(liq.size), ell, align = 'edge', tick_label = None, width = -0.97, label = 'Original Liquidity')
plt.bar(np.arange(liq.size), ell2-ell, align = 'edge', tick_label = None, width = -0.97, bottom = ell, label = 'Added Liquidity')
plt.xticks(ticks = np.arange(-1, liq.size), labels = liq.ticks, rotation = 45, ha = 'right')
plt.axvline(curr_er, color = 'red')
plt.title('Liquidity Distribution')
plt.xlabel('Exchange Rate')
plt.ylabel('Liquidity')
plt.legend()
plt.ylim(top = 150, bottom = 0)
plt.show()

In [ ]:
plt.close()
fig, axs = plt.subplots(2)
curr_er = liq.idx + (liq.pool_er - liq.ticks[liq.idx])/(liq.ticks[liq.idx + 1] - liq.ticks[liq.idx]) - 1

axs[0].bar(np.arange(liq.size), oldA, align = 'edge', tick_label = None, width = -0.97, label = 'Original Token A', alpha = 0.5)
axs[0].bar(np.arange(liq.size), liq.tokA - oldA, align = 'edge', tick_label = None, width = -0.97, bottom = oldA, label = 'Added Token A', alpha = 0.5)
axs[0].axvline(curr_er, color = 'red')
axs[0].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
axs[0].set_title('Token A Reserves (Increased)')
axs[0].set_ylabel('Quantity')
axs[0].set_ylim(top =8, bottom = 0)

axs[1].bar(np.arange(liq.size), oldB, align = 'edge', tick_label = None, width = -0.97, label = 'Original Token B', alpha = 0.5)
axs[1].bar(np.arange(liq.size), liq.tokB - oldB, align = 'edge', tick_label = None, width = -0.97, bottom = oldB, label = 'Removed Token B', alpha = 0.5)
axs[1].set_xticks(np.arange(-1, liq.size))
axs[1].set_xticklabels(liq.ticks, rotation = 45, ha = 'right')
axs[1].axvline(curr_er, color = 'red')
axs[1].set_title('Token B Reserves (Decreased)')
axs[1].set_xlabel('Exchange Rate')
axs[1].set_ylabel('Quantity')
axs[1].set_ylim(top = 12, bottom = 0)

fig.suptitle('Token Distributions after Liquidity Addition')
plt.show()

In [ ]:
liq.printDetails()

### Demonstration of Swap

In [ ]:
liq.resetLiquidity(variety, 1.6)

In [ ]:
liq.swap(-10)

In [ ]:
plt.close()
fig, axs = plt.subplots(2)
curr_er = liq.idx + (liq.pool_er - liq.ticks[liq.idx])/(liq.ticks[liq.idx + 1] - liq.ticks[liq.idx]) - 1

axs[0].bar(np.arange(liq.size), oldA, align = 'edge', tick_label = None, width = -0.97, label = 'Original Token A')
axs[0].bar(np.arange(liq.size), liq.tokA - oldA, align = 'edge', tick_label = None, width = -0.97, bottom = oldA, label = 'Added Token A')
axs[0].axvline(curr_er, color = 'red')
axs[0].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
axs[0].set_title('Token A Reserves')
axs[0].set_ylabel('Quantity')
axs[0].set_ylim(top =8, bottom = 0)
axs[0].legend()

axs[1].bar(np.arange(liq.size), oldB, align = 'edge', tick_label = None, width = -0.97, label = 'Original Token B')
axs[1].bar(np.arange(liq.size), liq.tokB - oldB, align = 'edge', tick_label = None, width = -0.97, bottom = oldB, label = 'Added Token B')
axs[1].set_xticks(np.arange(-1, liq.size))
axs[1].set_xticklabels(liq.ticks, rotation = 45, ha = 'right')
axs[1].axvline(curr_er, color = 'red')
axs[1].set_title('Token B Reserves')
axs[1].set_xlabel('Exchange Rate')
axs[1].set_ylabel('Quantity')
axs[1].set_ylim(top = 12, bottom = 0)
axs[1].legend()

fig.suptitle('Token Distributions after Liquidity Addition')
plt.show()

In [ ]:
liq.printDetails()

An example of how to generate random swap arrivals in the market and a visualization of them.

In [ ]:
swapper = Swapper(swapper_type = 1, swap_max = 10, swap_min = -10)
T = 10
time = np.arange(T)
swaps = np.zeros(T)
for t in range(T):
    x = swapper.generateSwaps((liq.mkt_er - liq.pool_er), 1)
    swaps[t] = x

In [ ]:
plt.close()
plt.stem(time, swaps)
plt.title('Exogenous Swap Transactions')
plt.xlabel('Time')
plt.ylabel('Swap Size')
plt.show()

### How do swaps and liquidity adjustments affect $\psi(x, \ell)$?

Showing that swaps do not change the shape of the exchange curve $\psi(x, v)$ (but can shift it, depending on how the calculations are done), for any given starting liquidity distribution.

In [ ]:
nu = Pool(pool_type = 1, mkt =1.5, pool =1.5, fee = 0.01)
q = np.ones(nu.size)*10
for i in range(nu.size):
    nu.updateLiq(i, q[i])
nu.printDetails()
a_list, b_list = nu.calculatePsi()

In [ ]:
A, r = nu.swap(10)
print(f'Swapped 10 of token B for {-A} of token A.')
nu.printDetails()
a_list2, b_list2 = nu.calculatePsi()

In [ ]:
# shift so they line up
print(np.amax(a_list2) - np.amax(a_list))
print(np.amax(b_list2) - np.amax(b_list))
a_list2 = np.sort(a_list2)-np.amax(a_list2) + np.amax(a_list)
b_list2 = np.sort(b_list2)-np.amax(b_list2) + np.amax(b_list)

In [ ]:
plt.close()
plt.plot(np.sort(b_list), np.sort(a_list), label = 'original')
plt.plot(b_list2, a_list2, label = 'shifted after swap')
plt.legend()
plt.show()

Compare the range and concavity of two graphs of $\psi$ for different initial deposits $v$.

In [ ]:
nu = Pool(pool_type = 1, mkt =1.5, pool =1.5, fee = 0.01)
q = np.ones(nu.size)*10
q[3]= 15 # increase the liquidity in one interval
for i in range(nu.size):
    nu.updateLiq(i, q[i])
nu.printDetails()
a_list3, b_list3 = nu.calculatePsi()

In [ ]:
print(f'The range of the uniformly distributed liquidity is {np.amin(a_list)} to {np.amax(a_list)}.')
print(f'The range of the concentrated liquidity is {np.amin(a_list3)} to {np.amax(a_list3)}.')

Observe that the graph with extra liquidity added in only one interval is both above than the original and less concave. It also has a larger domain. 

In [ ]:
plt.close()
#plt.plot(np.linspace(-35, 50), (1/1.69)*(np.linspace(-35, 50)- 7.94) +4.99)
plt.plot(np.sort(b_list), np.sort(a_list), label = 'Uniformly distributed liquidity')
plt.plot(np.sort(b_list3), np.sort(a_list3), label = 'Extra liquidity in interval 2')
plt.legend()
plt.xlabel('Amount of Token B deposited into the pool')
plt.ylabel('Amount of Token A withdrawn from the pool')
plt.show()

For a fixed $\ell$ and known $\xi$, it is easy to find the optimal Bot attack behavior.

In [ ]:
gas = 0.001

We want to ensure that the bot attack doesn't result in a price more than $s$% than expected, so we choose $x_1$ such that 
\begin{equation*}
\psi(x_1 + \xi, \ell) - \psi(x_1) = (1 - sign(\xi)*s) \psi(\xi, \ell).
\end{equation*}

Given a value for $x_1$ that is optimal, we can find the corresponding value of $x_2$ that will produce a symmetric swap by calculating
\begin{align*}
\psi(x_1 + \xi + x_2) =& \sum_{j=0}^i A_j + K_i/\sqrt{p^u_i} - \dfrac{K^2_i}{B_i + x_1 + \xi + x_2 - b_i + K_i\sqrt{p^l_i}}= \psi(x_1 + \xi, \ell) - \psi(x_1, \ell)\\
x_2 =& \dfrac{K_i^2}{A_i + K_i/\sqrt{p^u_i} - (\psi(x_1 + \xi, \ell) - \psi(x_1, \ell))} - (B_i + x_1 + \xi - b_i + K_i\sqrt{p^l_i}).
\end{align*}
Also note that for a given slippage percentage $s$, we have the constraint that $\psi(x_1 + \xi, \ell) - \psi(x_1, \ell) = (1 - s)\psi(\xi, \ell)$.

#### Implementation

Testing out the `bot_opt` and `calcProfit` functions on a toy example:

In [ ]:
pi = []

In [ ]:
l0 = np.array([10, 10,10])
l = np.array([10, 62,60])
m_star = 1
p_star = 1
gamma = 0.001

nu = Pool(pool_type = 0, mkt = m_star, pool = p_star, fee = gamma)
nu.resetLiquidity(l, p_star)
bot = Bot(thresh = 0, cap = 100, lam = 0)

In [ ]:
nu.printDetails()

In [ ]:
x_2 = symSwap(10, 50, nu)
nu.psi(10 + 50 + x_2)[0] - nu.psi(10 + 50)[0] + nu.psi(10)[0]

In [ ]:
if 1 == 3:
    print(2*gamma/m_star)
    breaks = np.concatenate((nu.b_ticks[:-1], nu.b_ticks[1:] - xi))
    breaks = breaks[breaks >= nu.b_ticks[0]-xi]
    breaks = breaks[breaks <= nu.b_ticks[-1]]
    breaks = np.sort(np.unique(breaks))

    if xi > 0:    
        breaks = np.unique(np.maximum(breaks, 0)) # make sure x1 > 0
    else:
        breaks = np.unique(np.minimum(breaks, xi)) # make sure x1 < 0

    for i in range(len(breaks)-1):
        print(f'Interval from {breaks[i]} to {breaks[i+1]}.')
        for dx in np.arange(breaks[i], breaks[i+1], 1):
            a1, r1 = nu.psi(dx)
            a2, r2 = nu.psi(dx + xi)
            diff = np.sign(xi)*(1/r1 - 1/r2)
            print(diff)
    nu.b_ticks

In [ ]:
avgB = 0
avgLP = 0

if p_star < m_star:
    prob = np.array([0.95, 0.05])
    xilist = np.array([5, -1])
elif p_star > m_star:
    prob = np.array([0.95, 0.05])
    xilist = np.array([-5, 1])
else:
    prob = np.array([0.5, 0.5])
    xilist = np.array([-3, 3])

for i in range(len(xilist)):
    xi = xilist[i]
    x_1, x_2 = bot.optimize(nu, xi)
    x_bar = x_1 + xi + x_2
    print(f'x_1 = {round(x_1, 5)} and x_2 = {round(x_2, 5)} and x_bar = {round(x_bar, 5)}.')

    # introduce a limit to the Bot's capital
    bot_capital = 100
    x_1 = min(x_1,  bot_capital)
    x_1 = max(x_1, -bot_capital)
    x_2 = x_bar - x_1 - xi

    pi_LP, pi_B = predictProfit(x_1, x_2, xi, nu)   
    pi_LP = pi_LP - 0.001*np.sum(np.abs(l - l0))
    print(f"The bot profit in terms of token B is {round(pi_B, 4)}.")
    print(f"The liquidity provider profit in terms of token B is {round(pi_LP, 4)}.")

    avgB += pi_B*prob[i]
    avgLP += pi_LP*prob[i]

In [ ]:
pi.append([avgB, avgLP])
pi

Sometimes I want to double-check that my analytical formulas match what I've programmed - I can run the below cell to test.

In [ ]:
# check psi(x_bar) computed value
p1, r = nu.psi(x_bar)
# check psi(x_bar) simplified formula
p2 = nu.tokA[nu.idx] + nu.K(nu.idx)/np.sqrt(nu.ticks[nu.idx+1]) - nu.K(nu.idx)*np.sqrt((1 - gamma)/m_star)
p2 = -p2
# check psi(x_bar) explicit formula
p3 = nu.tokA[nu.idx] + nu.K(nu.idx)/np.sqrt(nu.ticks[nu.idx+1]) - nu.K(nu.idx)**2/(nu.tokB[nu.idx] + x_bar + nu.K(nu.idx)*np.sqrt(nu.ticks[nu.idx]))
# check x_bar formula
p4 = -nu.tokB[nu.idx] - nu.K(nu.idx)*np.sqrt(nu.ticks[nu.idx]) + nu.K(nu.idx)*np.sqrt(m_star/(1-gamma))
print(f'{p1} = {p2} = {p3}?')
print(f'{x_bar} = {p4}?')